In [ ]:
from tqdm import tqdm
# from __future__ import print_function
# from __future__ import division
import torch
import pathlib2
from pathlib2 import Path
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import pandas as pd
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from src import constant
# from robustbench import load_model
from src.utils import get_model
import time
import glob
import os
import matplotlib.pyplot as plt
import copy
from src.bootstrap import Cifar10Bootstrapper
from src.job import Cifar10Job
from src.utils import get_transformation_threshold, transform_image, transform_image_dir
import cv2
from src.dataset import GeneralDataset, GeneralDatasetAlter
from src.utils import start_matlab
import seaborn as sns
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
%matplotlib inline

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "Kireev2021Effectiveness_Gauss50percent"
num_classes = 10
train_dataset_path = "./transformed_data/train"
val_dataset_path = "./transformed_data/val"
batch_size = 8
num_epochs = 10
feature_extract = False

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=25):
    model.eval()
    print(device)
    model.to(device)
    avg_loss, avg_acc, _ = eval_model(dataloaders['val'], model, criterion, device)
    print(f"initial performance: avg_loss: {avg_loss}, avg_acc: {avg_acc}")
    since = time.time()
    
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    loss = {'train': [], 'val': []}
    acc = {'train': [], 'val': []}
    performance_records = []
    result_records = []
    pbar = tqdm(total=num_epochs)
    for epoch in range(num_epochs):
        # print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        # print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels, IQA_scores, filepaths in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                # record correctness for each image and their IQA values
                corrects = preds == labels.data
                for k in range(len(corrects)):
                    result_records.append({
                        'filepath': str(filepaths[k]),
                        'is_correct': bool(corrects[k]),
                        'label': int(labels.data[k]),
                        'phase': phase,
                        'epoch': epoch,
                        'pred': int(preds[k]),
                        'IQA_score': float(IQA_scores[k])
                    })

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            pbar.set_postfix({'epoch': epoch,'phase': phase,'loss': epoch_loss, 'accuracy': float(epoch_acc)})
            # print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            # loss[phase].append(epoch_loss)
            # acc[phase].append(epoch_acc)
            performance_records.append({
                'epoch': epoch,
                'value': float(epoch_loss),
                'type': 'loss', 
                'phase': phase
            })
            performance_records.append({
                'epoch': epoch,
                'value': float(epoch_acc),
                'type': 'acc', 
                'phase': phase
            })
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
        pbar.update()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # plot
    # sns.lineplot(x="epoch", y="value",
    #          hue="type", style="phase",
    #          data=pd.DataFrame(data=performance_records))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, performance_records, result_records

In [ ]:
def eval_model(dataloader, model, criterion, device):
    model.eval()
    running_loss = 0
    running_corrects = 0
    result_records = []
    for inputs, labels, IQA_scores, filepaths in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        corrects = preds == labels.data
        for k in range(len(filepaths)):
            result_records.append({
                'filepath': str(filepaths[k]),
                'is_correct': bool(corrects[k]),
                'label': int(labels.data[k]),
                'pred': int(preds[k]),
                'IQA_score': float(IQA_scores[k])
            })
    avg_loss = running_loss / len(dataloader.dataset)
    avg_acc = running_corrects.double() / len(dataloader.dataset)
    return float(avg_loss), float(avg_acc), pd.DataFrame(data=result_records)

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
performance_results = []

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

# image_datasets = {
#     'train': GeneralDataset(pathlib2.Path("./transformed_data/train")),
#     'val': GeneralDataset(pathlib2.Path("./transformed_data/val"))
# }

image_datasets = {
    'train': GeneralDatasetAlter(Path("./transformed_data/train"), Path("./data/cifar10_pytorch/train"), 0.5),
    'val': GeneralDatasetAlter(Path("./transformed_data/val"), Path("./data/cifar10_pytorch/val"), 0.5)
}


# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = get_model(model_name)
set_parameter_requires_grad(model_ft, feature_extract)
model_ft = model_ft.to(device)

In [ ]:
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

In [ ]:
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_ft, hist, performance_records, result_records = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, device, num_epochs=num_epochs)

# Training Performance

In [ ]:
trained_performance_df = pd.DataFrame(performance_records)
trained_performance_df

# Training Progress

In [ ]:
sns.lineplot(x="epoch", y="value", style='phase', hue='type', data=trained_performance_df)

# Training Records

In [ ]:
result_df = pd.DataFrame(data=result_records)
result_df

# IQA Analysis

In [ ]:
df1 = result_df[result_df['phase']=='val'].sort_values(by=['IQA_score'])
df1['IQA_interval'] = round(df1['IQA_score'], 1)
df1

In [ ]:
df1.groupby(by=["IQA_interval"]).sum()

In [ ]:
trained_IQA_int_dist_df = df1[df1['epoch']==df1['epoch'].max()].groupby(by=["IQA_interval"]).sum()[['is_correct']]
trained_IQA_int_dist_df

# Finetuned Model + Transformed Dataset

In [ ]:
avg_loss, avg_acc, _ = eval_model(dataloaders_dict['val'], model_ft, criterion, device)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")
performance_results.append({'model': 'original', 'dataset': 'original', 'accuracy': avg_acc, 'loss': avg_loss})

# Original Model + Transformed Dataset (val)

In [ ]:
original_model_ft = get_model(model_name)
original_model_ft = original_model_ft.to(device)
original_model_ft.eval()
# evaluate
avg_loss, avg_acc, records_df = eval_model(dataloaders_dict['val'], original_model_ft, criterion, device)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")
performance_results.append({'model': 'original', 'dataset': 'transformed', 'accuracy': avg_acc, 'loss': avg_loss})

In [ ]:
records_df['IQA_interval'] = round(records_df['IQA_score'], 1)
records_df

In [ ]:
records_df.sort_values(by=['IQA_score']).groupby(by=['IQA_interval']).sum()
orig_IQA_interval_dist_df = records_df.sort_values(by=['IQA_score']).groupby(by=['IQA_interval']).sum()[['is_correct']]
orig_IQA_interval_dist_df

In [ ]:
orig_IQA_interval_dist_df.hist()

In [ ]:
trained_IQA_int_dist_df.hist()

In [ ]:
orig_IQA_interval_dist_df['type'] = 'original'
trained_IQA_int_dist_df['type'] = 'trained'
orig_IQA_interval_dist_df.reset_index(inplace=True)
trained_IQA_int_dist_df.reset_index(inplace=True)

In [ ]:
sns.barplot(x="IQA_interval", y="is_correct", hue="type", data=pd.concat([orig_IQA_interval_dist_df, trained_IQA_int_dist_df]))

In [ ]:
df = records_df.sort_values(by=['IQA_score'])
df.groupby(by='IQA_interval').count()

In [ ]:
df1[df1['epoch']==df1['epoch'].max()].groupby(by=["IQA_interval"]).count()

In [ ]:
orig_IQA_interval_dist_df['is_correct'].sum()

In [ ]:
trained_IQA_int_dist_df['is_correct'].sum()

# Original Model on Original Dataset

In [ ]:
original_dataset = GeneralDataset(pathlib2.Path("./data/cifar10_pytorch/val"))
original_dataloader = torch.utils.data.DataLoader(original_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
avg_loss, avg_acc, _ = eval_model(original_dataloader, original_model_ft, criterion, device)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")
performance_results.append({'model': 'original', 'dataset': 'original', 'accuracy': avg_acc, 'loss': avg_loss})

# New Model on Original Dataset

In [ ]:
avg_loss, avg_acc, _ = eval_model(original_dataloader, model_ft, criterion, device)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")
performance_results.append({'model': 'finetuned', 'dataset': 'original', 'accuracy': avg_acc, 'loss': avg_loss})

# Transformed Model on Cifar10C

In [ ]:
cifar10c_dataset = GeneralDataset(pathlib2.Path("./data/cifar-10-c-images/contrast"))
cifar10c_dataloader = torch.utils.data.DataLoader(cifar10c_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
avg_loss, avg_acc, _ = eval_model(cifar10c_dataloader, model_ft, criterion, device)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")
performance_results.append({'model': 'finetuned', 'dataset': 'cifar10c', 'accuracy': avg_acc, 'loss': avg_loss})

# Original Model on Cifar10C

In [ ]:
avg_loss, avg_acc, _ = eval_model(cifar10c_dataloader, original_model_ft, criterion, device)
print(f"avg_loss: {avg_loss}, avg_acc: {avg_acc}")
performance_results.append({'model': 'original', 'dataset': 'cifar10c', 'accuracy': avg_acc, 'loss': avg_loss})

In [ ]:
final_result_df = pd.DataFrame(data=performance_results)
final_result_df